In [4]:
pip install tensorflow keras

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade tensorflow keras

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install scikit-learn

In [12]:
import os
import pandas as pd
import numpy as np 
import itertools
import tensorflow as tf
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input, LeakyReLU
from tensorflow.keras import applications 
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.utils import to_categorical  
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import math  
import datetime  
import time 
from PIL import Image, UnidentifiedImageError
%matplotlib inline

In [14]:
# กำหนดขนาดภาพ
img_width, img_height = 224, 224  

# กำหนดเส้นทางสำหรับบันทึกน้ำหนักโมเดล
top_model_weights_path = 'D:/InceptionV3/InceptionV3_model.weights.h5'  

# สร้างไดเรกทอรีทั้งหมดที่จำเป็น ถ้ายังไม่มีอยู่
os.makedirs(os.path.dirname(top_model_weights_path), exist_ok=True)

# กำหนดเส้นทางไปยังชุดข้อมูล
train_data_dir = r'D:\InceptionV3\archive\dataset\dataset_updated\training_set' 
validation_data_dir = r'D:\InceptionV3\archive\dataset\dataset_updated\validation_set'  
test_data_dir = r'D:\InceptionV3\archive\musemart\dataset_updated\validation_set'

# กำหนดจำนวน epoch และ batch size
epochs = 30 
batch_size = 64 

In [16]:
print(f"Training data directory: {train_data_dir}")
print(f"Validation data directory: {validation_data_dir}")
print(f"Test data directory: {test_data_dir}")

Training data directory: D:\InceptionV3\archive\dataset\dataset_updated\training_set
Validation data directory: D:\InceptionV3\archive\dataset\dataset_updated\validation_set
Test data directory: D:\InceptionV3\archive\musemart\dataset_updated\validation_set


In [18]:
def validate_images(directory):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(valid_extensions):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # ตรวจสอบความถูกต้องของไฟล์
                except (IOError, SyntaxError, UnidentifiedImageError) as e:
                    print(f"Invalid image file detected: {file_path}. Removing file.")
                    os.remove(file_path)
            else:
                # ลบไฟล์ที่ไม่ใช่รูปภาพ
                file_path = os.path.join(root, file)
                print(f"Non-image file detected: {file_path}. Removing file.")
                os.remove(file_path)

def convert_images_to_rgb(directory):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(valid_extensions):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        # ตรวจสอบโหมดภาพ
                        if img.mode in ('P', 'RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
                            rgb_img = img.convert('RGB')
                            rgb_img.save(file_path)
                            print(f"Converted {file_path} to RGB")
                except (IOError, SyntaxError, UnidentifiedImageError) as e:
                    print(f"Could not convert {file_path}: {e}")
                    # อาจต้องการลบไฟล์ที่ไม่สามารถแปลงได้
                    os.remove(file_path)
            else:
                # ลบไฟล์ที่ไม่ใช่รูปภาพ
                file_path = os.path.join(root, file)
                print(f"Non-image file detected: {file_path}. Removing file.")
                os.remove(file_path)

def detailed_validate_images(directory):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    invalid_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(valid_extensions):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()
                        img = img.convert('RGB')  # แปลงเป็น RGB เพื่อให้แน่ใจ
                        img.save(file_path)
                except (IOError, SyntaxError, UnidentifiedImageError) as e:
                    print(f"Invalid image file detected and removed: {file_path}. Error: {e}")
                    invalid_files.append(file_path)
                    os.remove(file_path)
            else:
                # ลบไฟล์ที่ไม่ใช่รูปภาพ
                file_path = os.path.join(root, file)
                print(f"Non-image file detected and removed: {file_path}")
                os.remove(file_path)
    print(f"Total invalid or non-image files removed: {len(invalid_files)}")

def find_invalid_images(directory):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    invalid_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(valid_extensions):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        img.verify()
                except (IOError, SyntaxError, UnidentifiedImageError) as e:
                    print(f"Invalid image file: {file_path}. Error: {e}")
                    invalid_files.append(file_path)
    print(f"Found {len(invalid_files)} invalid images.")
    return invalid_files

def safe_flow_from_directory(directory, **kwargs):
    datagen = ImageDataGenerator(**kwargs)
    generator = datagen.flow_from_directory(directory, **kwargs)
    while True:
        try:
            batch = next(generator)
            yield batch
        except UnidentifiedImageError as e:
            print(f"Skipped an invalid image: {e}")
            continue
        except StopIteration:
            break

In [22]:
# โหลดโมเดล InceptionV3 โดยไม่รวมชั้นการจำแนกประเภท
inceptionV3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#inceptionV3.trainable = False
datagen = ImageDataGenerator(rescale=1. / 255)

In [26]:
start = datetime.datetime.now()

# สร้าง generator สำหรับการฝึก
generator_train = datagen.flow_from_directory(  
    train_data_dir,  
    target_size=(img_width, img_height),  
    batch_size=batch_size,  
    class_mode='categorical',  
    shuffle=False,
    color_mode='rgb')  

nb_train_samples = len(generator_train.filenames)  
num_classes = len(generator_train.class_indices)  

predict_size_train = int(math.ceil(nb_train_samples / batch_size))  

# สร้าง bottleneck features สำหรับการฝึกด้วย VGG19
bottleneck_features_train = inceptionV3.predict(generator_train, steps=predict_size_train, verbose=1)  

# บันทึกฟีเจอร์ที่สร้างขึ้นสำหรับ VGG19
np.save(r'D:\InceptionV3\bottleneck_features_train.npy', bottleneck_features_train)

# จบจับเวลา
end = datetime.datetime.now()
elapsed = end - start
print('Time for train bottleneck features:', elapsed)

Found 7721 images belonging to 5 classes.


D:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 95/121 ━━━━━━━━━━━━━━━━━━━━ 1:49 4s/step

D:\anaconda\Lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


121/121 ━━━━━━━━━━━━━━━━━━━━ 530s 4s/step
Time for train bottleneck features: 0:10:27.053958


In [32]:
# เริ่มต้นจับเวลาใหม่สำหรับ validation
start = datetime.datetime.now()

# สร้าง generator สำหรับ validation
generator_val = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=False,
    color_mode='rgb'
)

nb_validation_samples = len(generator_val.filenames)
print(f"Number of validation samples: {nb_validation_samples}")

predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

# สร้าง bottleneck features สำหรับ validation ด้วย InceptionV3
bottleneck_features_validation = inceptionV3.predict(generator_val, steps=predict_size_validation ,verbose=1)

np.save(r'D:\InceptionV3\bottleneck_features_validation.npy', bottleneck_features_validation) 

# จบจับเวลา
end = datetime.datetime.now()
elapsed = end - start
print('Time for validation bottleneck features:', elapsed)

Found 856 images belonging to 5 classes.
Number of validation samples: 856
14/14 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step
Time for validation bottleneck features: 0:01:08.823763


In [ ]:
# เริ่มต้นจับเวลาใหม่สำหรับ test
start = datetime.datetime.now()

# สร้าง generator สำหรับ test
generator_test = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=False,
    color_mode='rgb'
)

# Count number of test samples
nb_test_samples = len(generator_test.filenames)
print(f"Number of test samples: {nb_test_samples}")

predict_size_test = int(math.ceil(nb_test_samples / batch_size))

# Generate bottleneck features for test ด้วย InceptionV3
try:
    bottleneck_features_test = inceptionV3.predict(generator_test, steps=predict_size_test, verbose=1)
    np.save(r'D:\InceptionV3\bottleneck_features_test.npy', bottleneck_features_test)
except UnidentifiedImageError as e:
    print(f"Error during test bottleneck features generation: {e}")

# จบจับเวลา
end = datetime.datetime.now()
elapsed = end - start
print('Time for test bottleneck features:', elapsed)

Found 265 images belonging to 5 classes.
Number of test samples: 265
2/5 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step 

In [ ]:
train_data = np.load(r'D:\InceptionV3\bottleneck_features_train.npy')
train_labels = to_categorical(generator_train.classes, num_classes=num_classes)
validation_data = np.load(r'D:\InceptionV3\bottleneck_features_validation.npy')
validation_labels = to_categorical(generator_val.classes, num_classes=num_classes)

In [ ]:
#Model summary
model.summary()

In [ ]:
# Clear memory
import gc
gc.collect()

# เริ่มต้นจับเวลา
start = datetime.datetime.now()

# ตรวจสอบขนาดของข้อมูล
print("train_data shape:", train_data.shape)  
print("train_labels shape:", train_labels.shape)  

# ตรวจสอบจำนวนตัวอย่างที่ไม่ซ้ำใน labels
unique_labels = np.unique(train_labels)
print("Unique labels:", unique_labels)


if train_data.shape[0] != train_labels.shape[0]:
    print("Adjusting train_labels to match train_data...")
    train_labels = train_labels[:train_data.shape[0]]

# สร้างโมเดล
model = Sequential()
model.add(Input(shape=train_data.shape[1:]))  # ใช้ Input แทน Flatten
model.add(Flatten())
model.add(Dense(100))
model.add(LeakyReLU(negative_slope=0.3))  # เปลี่ยน alpha เป็น negative_slope
model.add(Dropout(0.5))  
model.add(Dense(50))
model.add(LeakyReLU(negative_slope=0.3))  # เปลี่ยน alpha เป็น negative_slope
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))  

# คอมไพล์โมเดล
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)  

# ฝึกโมเดล
history = model.fit(
    train_data, 
    train_labels,  
    epochs=7,
    batch_size=batch_size,  
    validation_data=(validation_data, validation_labels)
)  

# บันทุนน้ำหนักโมเดล
model.save_weights(top_model_weights_path)  

# ประเมินโมเดล
eval_loss, eval_accuracy = model.evaluate(
    validation_data, 
    validation_labels, 
    batch_size=batch_size, 
    verbose=1
)

# แสดงผลลัพธ์
print("[INFO] Accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))  

# จบการจับเวลา
end = datetime.datetime.now()
elapsed = end - start
print('Time Elapsed:', elapsed) 

In [ ]:
# การวาดกราฟการฝึก
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.plot(epochs_range, acc, 'r', label='Training accuracy')
plt.plot(epochs_range, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.ylabel('Accuracy')  
plt.xlabel('Epoch')
plt.legend()
plt.figure()
plt.plot(epochs_range, loss, 'r', label='Training loss')
plt.plot(epochs_range, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('Loss')  
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Ensure that you load bottleneck features for the test set
# Assuming you already have 'bottleneck_features_test_inceptionV3.npy' generated
test_data = np.load(r'D:\InceptionV3\bottleneck_features_test.npy')

# Convert test labels to categorical
test_labels = to_categorical(generator_test.classes, num_classes=num_classes)

# Now you can evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size, verbose=1)
print("[INFO] Test Accuracy: {:.2f}%".format(test_accuracy * 100))  
print("[INFO] Test Loss: {}".format(test_loss))

In [ ]:
print('test data', test_data)
preds = np.round(model.predict(test_data),0) 
#to fit them into classification metrics and confusion metrics, some additional modificaitions are required
print('rounded test_labels', preds)

In [ ]:
# Generate classification report
art = ['drawings', 'engraving', 'iconography', 'painting', 'sculpture']
classification_metrics = classification_report(
    categorical_test_labels, 
    categorical_preds, 
    target_names=art,
    zero_division=0
)
print(classification_metrics)

In [ ]:
#Since our data is in dummy format we put the numpy array into a dataframe and call idxmax axis=1 to return the column
# label of the maximum value thus creating a categorical variable
#Basically, flipping a dummy variable back to it's categorical variable
categorical_test_labels = pd.DataFrame(test_labels).idxmax(axis=1)
categorical_preds = pd.DataFrame(preds).idxmax(axis=1)
confusion_matrix= confusion_matrix(categorical_test_labels, categorical_preds)

In [ ]:
#To get better visual of the confusion matrix:
def plot_confusion_matrix(cm, classes,
             normalize=False,
             title='Confusion matrix',
             cmap=plt.cm.Blues):
    #Add Normalization Option
    '''prints pretty confusion metric with normalization option '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
#     print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plot_confusion_matrix(confusion_matrix, ['drawings', 'engraving', 'iconography', 'painting', 'sculpture'])

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Convert one-hot encoded labels back to class labels
test_labels_flat = np.argmax(test_labels, axis=1)
preds_flat = np.argmax(preds, axis=1)

# Compute confusion matrix
cm = confusion_matrix(test_labels_flat, preds_flat, labels=[0, 1, 2, 3, 4])

# Normalize the confusion matrix
def normalize_cm(cm):
    with np.errstate(divide='ignore', invalid='ignore'):
        normalized_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    normalized_cm[np.isnan(normalized_cm)] = 0
    return normalized_cm

normalized_cm = normalize_cm(cm)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
ConfusionMatrixDisplay(normalized_cm, display_labels=[' drawings ', ' engraving   ', '   iconography', '   painting', ' sculpture ']).plot(cmap=plt.cm.Blues)
plt.title('Normalized Confusion Matrix')
plt.show()

In [ ]:
# ฟังก์ชันสำหรับอ่านและเตรียมภาพเดี่ยว
def read_image(file_path):
    print("[INFO] Loading and preparing image...")
    image = load_img(file_path, target_size=(224, 224))  
    image = img_to_array(image)  
    image = np.expand_dims(image, axis=0)
    image /= 255.  
    return image

def test_single_image(path):
    art = ['drawings', 'engraving', 'iconography', 'painting', 'sculpture']
    images = read_image(path)
    time.sleep(0.5)
    bt_prediction = vgg19.predict(images)  
    preds = model.predict(bt_prediction)

    for idx, label, x in zip(range(len(art)), art, preds[0]):
        print(f"ID: {idx}, Label: {label}, ความมั่นใจ: {round(x * 100, 2)}%")

    print('Final Decision:')
    time.sleep(0.5)
    for x in range(3):
        print('.' * (x + 1))
        time.sleep(0.2)
    
    # Using a predefined class dictionary
    class_dictionary = {
        0: 'drawings',
        1: 'engraving',
        2: 'iconography',
        3: 'painting',
        4: 'sculpture'
    }

    # Get the predicted class
    class_predicted = np.argmax(preds, axis=1)  # Get the index of the max probability
    print("ID: {}, Label: {}".format(class_predicted[0], class_dictionary[class_predicted[0]]))  
    return load_img(path)

# Example path (make sure to update this with your actual image path)
path = r'D:\InceptionV3\picture_test.jpg'
test_single_image(path)